In [1]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM

c:\Users\jonas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Adding Special Tokens to tokenizer

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")
tokenizer.add_special_tokens({"pad_token": "<PAD>"})

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


1

In [3]:
inst = "</INST>"
inst_id = tokenizer(inst, return_tensors="pt")
inst_id

{'input_ids': tensor([[    1,  1533, 25580, 29958]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [64]:
# special_tokens_dict = {"additional_special_tokens": ["<INST>", "</INST>"]}
# tokenizer.add_special_tokens(special_tokens_dict)

2

In [4]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<PAD>']

In [6]:
test = "Eu quero que você me gere uma música brasileira"
input_ids = tokenizer(test, return_tensors="pt")
input_ids

{'input_ids': tensor([[    1, 16430,   439,  1489,   712,  7931, 30037,   592,   330,   406,
          3672, 15433, 22905,  3055]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
tokenizer.decode(list(input_ids['input_ids'][0]), skip_special_tokens=True)

'Eu quero que você me gere uma música brasileira'

In [101]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<PAD>', '<INST>', '</INST>']

# Loading Dataset

In [8]:
import datasets

dataset = datasets.load_dataset("dominguesm/Canarim-Instruct-PTBR-Dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1519
    })
})

In [9]:
dataset['train'][1234]

{'instruction': 'Nomeie as capitais dos três países seguintes',
 'input': 'ndia, Canadá, Egito',
 'output': 'ndia: Nova Deli. Canadá: Ottawa. Egito: Cairo.'}

In [10]:
def format_prompt_input(row):
    row['prompt'] = \
        f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.\n\n### Instrução:\n{row["instruction"]}\n\n### Entrada:\n{row['input']}\n\n### Resposta:\n"""

    return row

def format_prompt_no_input(row):
    row['prompt'] = \
        f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.\n\n### Instrução:\n{row["instruction"]}\n\n### Resposta:\n"""

    return row

def create_prompt(row):
    return format_prompt_no_input(row) if row['input'] == "" else format_prompt_input(row)

In [11]:
prompted_dataset = dataset.map(create_prompt)

In [12]:
prompted_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 1519
    })
})

In [13]:
print(prompted_dataset['train'][1234]['prompt'])

Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instrução:
Nomeie as capitais dos três países seguintes

### Entrada:
ndia, Canadá, Egito

### Resposta:



In [14]:
print(prompted_dataset['train'][0]['prompt'])

Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instrução:
Dê três dicas para se manter saudável.

### Resposta:



In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1519
    })
})

In [19]:
def add_eos_to_output(row):
    row['output'] += '</s>'
    return row

def prompt2id(row, tokenizer: LlamaTokenizer):
    row['input_ids'] = tokenizer(row['prompt'], padding="max_length",
                                max_length=2048, return_tensors="pt")
    
    return row

prompted_dataset = prompted_dataset.map(add_eos_to_output)
prompted_dataset = prompted_dataset.map(lambda row: prompt2id(row, tokenizer))
prompted_dataset

Map: 100%|██████████| 1519/1519 [00:01<00:00, 1272.19 examples/s]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids'],
        num_rows: 1519
    })
})

In [27]:
print(prompted_dataset['train'][0]['input_ids']['input_ids'])

[[1, 1976, 29874, 861, 29877, 7919, 3672, 5778, 2340, 712, 553, 1037, 345, 3672, 260, 598, 5444, 29889, 3423, 1037, 1564, 3672, 620, 27363, 712, 26534, 3357, 19967, 3425, 2503, 263, 26978, 2028, 2340, 29889, 13, 13, 2277, 29937, 2799, 582, 2340, 29901, 13, 29928, 30037, 26299, 270, 5070, 1702, 409, 767, 357, 872, 566, 28691, 29889, 13, 13, 2277, 29937, 2538, 27363, 29901, 13, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000

In [33]:
prompted_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids'],
        num_rows: 1519
    })
})

In [34]:
#prompted_dataset.save_to_disk("./custom_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 1519/1519 [00:00<00:00, 3208.56 examples/s]
